## Interactive Cropping Tool Demo

In [4]:
from interactivecrop.interactivecrop import main as crop
from interactivecrop.samples import sample_images, sample_names

crop(sample_images,sample_names)

GridBox(children=(Dropdown(description='Img Name:', layout=Layout(grid_area='im_selector', width='100%'), opti…